In [ ]:
# Gerekli Kütüphaneleri İçe Aktarma
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn (Modelleme Kütüphanesi)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# --- 1. Veri Yükleme ve Ön Hazırlık ---

# Veri dosyasını yüklemeyi dene
try:
    df = pd.read_csv("data/df_analiz_verisi.csv")
    print("Gerçek Veri Seti Başarıyla Yüklendi.")

except FileNotFoundError:
    print("UYARI: 'df_analiz_verisi.csv' bulunamadı. Sentetik Veri Oluşturuluyor.")
    
    # 150 gözlemli sentetik veri seti oluşturma
    np.random.seed(42)
    df = pd.DataFrame({
        'Guven_Puani': np.random.randint(2, 6, 150),
        'Etik_Risk': np.random.randint(1, 5, 150),
        'Benimseme_Niyeti': np.random.randint(3, 6, 150),
        'Kullanim_Sikligi': np.random.randint(1, 6, 150),
        'Yeterlilik': np.random.randint(2, 6, 150),
        'Motivasyon': np.random.randint(3, 6, 150),
        'Kullandi_ChatGPT': np.random.randint(0, 2, 150)
    })
    # Oluşturulan sentetik veriyi kaydedelim (tekrar kullanılmak üzere)
    # df.to_csv("data/df_analiz_verisi.csv", index=False)
    # print("Sentetik veri 'data/' klasörüne kaydedildi.")


print("\n--- Veri Setinin İlk Gözlemleri ---")
print(df.head())
print(f"\nToplam Gözlem Sayısı: {df.shape[0]}")

In [ ]:
# Bağımsız değişkenleri seçelim
X = df[['Guven_Puani', 'Etik_Risk', 'Yeterlilik', 'Motivasyon', 'Kullanim_Sikligi', 'Kullandi_ChatGPT']]

# --- Veri Ölçeklendirme ---
# Standard Scaler (Ortalama=0, Standart Sapma=1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\nVeri Ölçeklendirmesi Tamamlandı. Veriler Modellemeye Hazır.")